In [10]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size= 0.2, random_state= 42)

# 검증 세트 만들기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size= 0.2, random_state=42)

print(sub_input.shape, test_input.shape, val_input.shape)

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state= 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target), dt.score(test_input, test_target), dt.score(val_input, val_target))

(4157, 3) (1300, 3) (1040, 3)
0.9971133028626413 0.8569230769230769 0.864423076923077


In [11]:
# 교차 검증 : cross_validate(), 기본적으로 5-폴드 교차 검증 수행
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)
# fit_time : 모델을 훈련하는 시간, score_time : 모델을 검증하는 시간, test_score : 검증 폴드 점수

import numpy as np
print(np.mean(scores['test_score']))

{'fit_time': array([0.00900197, 0.01000357, 0.00800204, 0.01000237, 0.00784636]), 'score_time': array([0.00200057, 0.00099897, 0.00099993, 0.        , 0.00100255]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


In [12]:
# cross_validate는 훈련 세트를 섞어주지 않으므로 분할기를 사용해야 함
# 회귀모델 : KFold, 분류모델 : StratifiedKFold
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

# 10-폴드교차 검증과 훈련세트 섞기
splitter = StratifiedKFold(n_splits= 10, shuffle=True, random_state=42) # n_spits를 통해 몇 폴드 교차검증을 할 지 정할 수 있다
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.855300214703487
0.8574181117533719


In [13]:
# GridSearchCV : 하이퍼파라미터 탐색과 교차 검증을 한번에 수행
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs= 1) # n_jobs = 병렬 실행에 사용할 CPU 코어 수
gs.fit(train_input, train_target)

dt = gs.best_estimator_ # 검증 점수가 가장 높은 매개변수 조합
print(dt.score(train_input, train_target))

print(gs.best_params_) # 서치로 찾은 최적의 매개변수

print(gs.cv_results_['mean_test_score']) # 각 매개변수에서 수행한 교차검증의 평균 점수

# 넘파이 argmax() : 가장 큰 값의 인덱스 추출
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [14]:
# 더 복잡한 매개변수 조합 탐색
params = {
'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
'max_depth' : range(5,20, 1),
'min_samples_split' : range(2, 200, 10)
}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs= -1)
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [15]:
# 최상의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [16]:
# 랜덤 서치 : 매개 변수를 샘플링할 수 있는 확률 분포 객체를 전달
from scipy.stats import uniform, randint
# 두 클래스 모두 주어진 범위에서 값을 고르게 뽑음, uniform : 실수값, randint : 정수값
rgen = randint(0,10)
print(np.unique(rgen.rvs(1000), return_counts= True)) # unique() : 2차원 배열을 1차원으로 정렬, return_counts : 각 값의 개수
ugen = uniform(0,1)
print(ugen.rvs(10))

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([100, 100, 100, 115,  92,  95,  89,  92, 111, 106], dtype=int64))
[0.17515615 0.95843347 0.06125874 0.52577869 0.18758182 0.15235108
 0.22912906 0.15102314 0.00146216 0.50890807]


In [26]:
params = {
'min_impurity_decrease' : uniform(0.0001, 0.001),
'max_depth' : randint(20,50),
'min_samples_split' : randint(2, 25),
'min_samples_leaf' : randint(1, 25)
}

from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter= 100, n_jobs= -1, random_state= 42)
# n_iter= : 샘플링 횟수
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [27]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
